In [ ]:
from pathlib import Path

import pkg_resources as pkg

PATH_DEEPRANK_CORE = Path(pkg.resource_filename("deeprank2", ""))
ROOT = PATH_DEEPRANK_CORE.parent
PATH_TEST = ROOT / "tests"
import glob
import os

import h5py
import pandas as pd

from deeprank2.dataset import save_hdf5_keys
from deeprank2.domain.aminoacidlist import alanine, phenylalanine
from deeprank2.query import (
    ProteinProteinInterfaceQuery,
    QueryCollection,
    SingleResidueVariantQuery,
)
from deeprank2.tools.target import compute_ppi_scores

- Generating 1ATN_ppi.hdf5


In [ ]:
import warnings

from Bio import BiopythonWarning

from deeprank2.utils.grid import GridSettings, MapMethod

with warnings.catch_warnings():
    warnings.simplefilter("ignore", BiopythonWarning)
    warnings.simplefilter("ignore", RuntimeWarning)

    ref_path = str(PATH_TEST / "data/ref/1ATN/1ATN.pdb")
    pssm_path1 = str(PATH_TEST / "data/pssm/1ATN/1ATN.A.pdb.pssm")
    pssm_path2 = str(PATH_TEST / "data/pssm/1ATN/1ATN.B.pdb.pssm")
    chain_id1 = "A"
    chain_id2 = "B"
    pdb_paths = [
        str(PATH_TEST / "data/pdb/1ATN/1ATN_1w.pdb"),
        str(PATH_TEST / "data/pdb/1ATN/1ATN_2w.pdb"),
        str(PATH_TEST / "data/pdb/1ATN/1ATN_3w.pdb"),
        str(PATH_TEST / "data/pdb/1ATN/1ATN_4w.pdb"),
    ]

    queries = QueryCollection()

    for pdb_path in pdb_paths:
        # Append data points
        targets = compute_ppi_scores(pdb_path, ref_path)
        queries.add(
            ProteinProteinInterfaceQuery(
                pdb_path=pdb_path,
                resolution="residue",
                chain_ids=[chain_id1, chain_id2],
                targets=targets,
                pssm_paths={chain_id1: pssm_path1, chain_id2: pssm_path2},
            ),
        )

    # Generate graphs and save them in hdf5 files
    output_paths = queries.process(
        cpu_count=1,
        prefix="1ATN_ppi",
        grid_settings=GridSettings([20, 20, 20], [20.0, 20.0, 20.0]),
        grid_map_method=MapMethod.GAUSSIAN,
    )

- Generating residue.hdf5


In [ ]:
# Local data
project_folder = "/home/dbodor/git/DeepRank/deeprank-core/tests/data/sample_25_07122022/"
csv_file_name = "BA_pMHCI_human_quantitative.csv"
models_folder_name = "exp_nmers_all_HLA_quantitative"
data = "pMHCI"
resolution = "residue"  # either 'residue' or 'atom'
influence_radius = 15  # max distance in Å between two interacting residues/atoms of two proteins
max_edge_length = 15  # max distance in Å between to create an edge

csv_file_path = f"{project_folder}data/external/processed/I/{csv_file_name}"
models_folder_path = f"{project_folder}data/{data}/features_input_folder/{models_folder_name}"

pdb_files = glob.glob(os.path.join(models_folder_path + "/pdb", "*.pdb"))
pdb_files.sort()
print(f"{len(pdb_files)} pdbs found.")
pssm_m = glob.glob(os.path.join(models_folder_path + "/pssm", "*.M.*.pssm"))
pssm_m.sort()
print(f"{len(pssm_m)} MHC pssms found.")
pssm_p = glob.glob(os.path.join(models_folder_path + "/pssm", "*.P.*.pssm"))
pssm_p.sort()
print(f"{len(pssm_p)} peptide pssms found.")
csv_data = pd.read_csv(csv_file_path)
csv_data.cluster = csv_data.cluster.fillna(-1)
pdb_ids_csv = [pdb_file.split("/")[-1].split(".")[0].replace("-", "_") for pdb_file in pdb_files]
clusters = [csv_data[pdb_id == csv_data.ID].cluster.to_numpy()[0] for pdb_id in pdb_ids_csv]
bas = [csv_data[pdb_id == csv_data.ID].measurement_value.to_numpy()[0] for pdb_id in pdb_ids_csv]

queries = QueryCollection()
print(f"Adding {len(pdb_files)} queries to the query collection ...")
for i in range(len(pdb_files)):
    queries.add(
        ProteinProteinInterfaceQuery(
            pdb_path=pdb_files[i],
            resolution="residue",
            chain_ids=["M", "P"],
            influence_radius=influence_radius,
            max_edge_length=max_edge_length,
            targets={
                "binary": int(float(bas[i]) <= 500),  # binary target value
                "BA": bas[i],  # continuous target value
                "cluster": clusters[i],
            },
            pssm_paths={"M": pssm_m[i], "P": pssm_p[i]},
        ),
    )
print("Queries created and ready to be processed.\n")

output_paths = queries.process(prefix="residue")
print(output_paths)

- Generating train.hdf5, valid.hdf5, test.hdf5


In [ ]:
# dividing hdf5 file in train, valid, test
hdf5_path = "residue.hdf5"
train_clusters = [3, 4, 5, 2]
val_clusters = [1, 8]
test_clusters = [6]
target = "target_values"
feature = "cluster"

clusters = {}
train_ids = []
val_ids = []
test_ids = []

with h5py.File(hdf5_path, "r") as hdf5:
    for key in hdf5:
        feature_value = float(hdf5[key][target][feature][()])
        if feature_value in train_clusters:
            train_ids.append(key)
        elif feature_value in val_clusters:
            val_ids.append(key)
        elif feature_value in test_clusters:
            test_ids.append(key)

        if feature_value in clusters:
            clusters[int(feature_value)] += 1
        else:
            clusters[int(feature_value)] = 1

    print(f"Trainset contains {len(train_ids)} data points, {round(100*len(train_ids)/len(hdf5.keys()), 2)}% of the total data.")
    print(f"Validation set contains {len(val_ids)} data points, {round(100*len(val_ids)/len(hdf5.keys()), 2)}% of the total data.")
    print(f"Test set contains {len(test_ids)} data points, {round(100*len(test_ids)/len(hdf5.keys()), 2)}% of the total data.\n")

    for key, value in dict(sorted(clusters.items(), key=lambda x: x[1], reverse=True)).items():
        print(f"Group with value {key}: {value} data points, {round(100*value/len(hdf5.keys()), 2)}% of total data.")

save_hdf5_keys(hdf5_path, train_ids, "train.hdf5", hardcopy=True)
save_hdf5_keys(hdf5_path, val_ids, "valid.hdf5", hardcopy=True)
save_hdf5_keys(hdf5_path, test_ids, "test.hdf5", hardcopy=True)

- Generating variants.hdf5


In [ ]:
count_queries = 5
pdb_path = str(PATH_TEST / "data/pdb/3C8P/3C8P.pdb")
ref_path = str(PATH_TEST / "data/ref/3C8P/3C8P.pdb")
targets = compute_ppi_scores(pdb_path, ref_path)
queries = QueryCollection()

for number in range(1, count_queries + 1):
    query = SingleResidueVariantQuery(
        pdb_path=pdb_path,
        resolution="residue",
        chain_ids="A",
        variant_residue_number=number,
        insertion_code=None,
        wildtype_amino_acid=alanine,
        variant_amino_acid=phenylalanine,
        pssm_paths={"A": str(PATH_TEST / "data/pssm/3C8P/3C8P.A.pdb.pssm"), "B": str(PATH_TEST / "data/pssm/3C8P/3C8P.B.pdb.pssm")},
        targets=targets,
    )
    queries.add(query)

output_paths = queries.process(cpu_count=1, prefix="variants")

- Generating atom.hdf5


In [ ]:
ref_path = str(PATH_TEST / "data/ref/1ATN/1ATN.pdb")
pssm_path1 = str(PATH_TEST / "data/pssm/1ATN/1ATN.A.pdb.pssm")
pssm_path2 = str(PATH_TEST / "data/pssm/1ATN/1ATN.B.pdb.pssm")
chain_id1 = "A"
chain_id2 = "B"
pdb_paths = [
    str(PATH_TEST / "data/pdb/1ATN/1ATN_1w.pdb"),
    str(PATH_TEST / "data/pdb/1ATN/1ATN_2w.pdb"),
    str(PATH_TEST / "data/pdb/1ATN/1ATN_3w.pdb"),
    str(PATH_TEST / "data/pdb/1ATN/1ATN_4w.pdb"),
]

queries = QueryCollection()

for pdb_path in pdb_paths:
    # Append data points
    targets = compute_ppi_scores(pdb_path, ref_path)
    queries.add(
        ProteinProteinInterfaceQuery(
            pdb_path=pdb_path,
            resolution="atom",
            chain_ids=[chain_id1, chain_id2],
            targets=targets,
            pssm_paths={chain_id1: pssm_path1, chain_id2: pssm_path2},
        ),
    )

# Generate graphs and save them in hdf5 files
output_paths = queries.process(cpu_count=1, prefix="atom")